In [1]:
# Notebook for investigating breakthrough infections and comparing with total country-wide deaths
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define datapath and load
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"\\" + ssidatapath


# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
tuePath = 'overvaagningsdata-covid19-14092021-jrhb'
# tuePath = 'SSI_data_2021-09-14'
# tuePath = 'SSI_data_2021-09-28'
tuePath = 'SSI_data_2021-10-05'

dirPath = rootdir + '\\' + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')
df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';')
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';')
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';')

In [143]:
# Load population count
dkpopdf = pd.read_excel('DKfolketal2021clean.xlsx')

In [121]:
# Load the data from Danmark Statistik
allDeathDF = pd.read_csv('DKstatDodc2021_clean.csv',delimiter=';',encoding='latin-1')
# allDeathDF.transpose()

# Get dates
allDatesDK = np.array([np.datetime64(str(x[0:4])+'-'+str(x[5:7])+'-'+str(x[8:10])) for x in allDeathDF.columns[1:]])

allDeathDF = allDeathDF.set_index('Age').transpose()
allDeathDF.index = allDatesDK

In [99]:
fig,ax1 = plt.subplots()

meanWidth = 14
print(allDeathDF.columns)
ax1.plot(rnTime(allDatesDK,meanWidth),rnMean(allDeathDF['85-89 år'],meanWidth))
ax1.plot(rnTime(allDatesDK,meanWidth),rnMean(allDeathDF['90-94 år'],meanWidth))
ax1.plot(rnTime(allDatesDK,meanWidth),rnMean(allDeathDF['95-99 år'],meanWidth))
ax1.plot(rnTime(allDatesDK,meanWidth),rnMean(allDeathDF['100 år og derover'],meanWidth))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index(['0-4 år', '5-9 år', '10-14 år', '15-19 år', '20-24 år', '25-29 år',
       '30-34 år', '35-39 år', '40-44 år', '45-49 år', '50-54 år', '55-59 år',
       '60-64 år', '65-69 år', '70-74 år', '75-79 år', '80-84 år', '85-89 år',
       '90-94 år', '95-99 år', '100 år og derover'],
      dtype='object', name='Age')


# Flip dataframes

In [102]:
# Flip dataframes to be like df1
casedf = pd.DataFrame()
casedf_once = pd.DataFrame()
casedf_both = pd.DataFrame()
casedf_full = pd.DataFrame()
casedf_none = pd.DataFrame()

weekNames = df1.Ugenummer
casedf['Ugenummer'] = weekNames
casedf_none['Ugenummer'] = weekNames
casedf_once['Ugenummer'] = weekNames
casedf_both['Ugenummer'] = weekNames
casedf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_C)):
    curRow = df2_C.loc[k]
    curAge = df2_C.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    casedf[curAge+'_Ingen vaccination'] = curNone
    casedf[curAge+'_Første vaccination'] = curOnce
    casedf[curAge+'_Anden vaccination'] = curBoth 
    casedf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    casedf_none[curAge] = curNone
    casedf_once[curAge] = curOnce
    casedf_both[curAge] = curBoth 
    casedf_full[curAge] = curFull
    
casedf

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 39,162,13,16,14,76,4,28,28,230,...,119,119,4,2,51,51,931,90,1173,1163
1,Uge 38,171,14,8,6,57,7,33,33,209,...,128,128,3,0,75,75,812,77,964,951
2,Uge 37,148,12,4,3,61,3,14,12,208,...,83,83,2,0,48,48,752,74,781,769
3,Uge 36,183,29,16,8,111,5,38,34,311,...,116,116,3,0,67,67,1130,173,1101,1066
4,Uge 35,330,75,25,15,131,24,53,52,404,...,115,115,11,2,68,67,1580,271,1475,1407
5,Uge 34,493,126,37,10,221,33,72,68,602,...,148,148,9,1,64,64,2310,471,1699,1565
6,Uge 33,590,155,30,1,290,41,99,83,799,...,109,109,3,0,63,63,2751,664,1661,1407
7,Uge 32,483,173,17,1,336,73,122,85,1056,...,101,100,7,2,46,46,2928,1075,1569,1210
8,Uge 31,444,102,6,0,341,138,130,64,991,...,70,70,0,0,36,36,2773,1353,1324,992
9,Uge 30,344,68,0,0,288,271,146,52,1007,...,60,60,1,0,24,24,2422,1946,1200,803


In [103]:
# Flip dataframe to be like df1
hospdf = pd.DataFrame()
hospdf_once = pd.DataFrame()
hospdf_both = pd.DataFrame()
hospdf_full = pd.DataFrame()
hospdf_none = pd.DataFrame()

weekNames = df1.Ugenummer
hospdf['Ugenummer'] = weekNames
hospdf_none['Ugenummer'] = weekNames
hospdf_once['Ugenummer'] = weekNames
hospdf_both['Ugenummer'] = weekNames
hospdf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_H)):
    curRow = df2_H.loc[k]
    curAge = df2_H.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    hospdf[curAge+'_Ingen vaccination'] = curNone
    hospdf[curAge+'_Første vaccination'] = curOnce
    hospdf[curAge+'_Anden vaccination'] = curBoth 
    hospdf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    hospdf_none[curAge] = curNone
    hospdf_once[curAge] = curOnce
    hospdf_both[curAge] = curBoth 
    hospdf_full[curAge] = curFull
    

# df2_H
# curRow
hospdf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 39,0,0,1,1,0,1,0,0,8,...,15,15,1,1,12,12,35,4,43,43
1,Uge 38,0,0,0,0,1,0,0,0,3,...,17,17,2,0,24,24,30,2,54,54
2,Uge 37,0,0,0,0,0,0,0,0,4,...,11,11,0,0,18,18,34,5,44,44
3,Uge 36,1,0,0,0,0,0,0,0,19,...,14,14,3,0,20,20,79,9,59,58
4,Uge 35,4,0,0,0,3,0,0,0,18,...,12,12,9,2,18,17,120,13,47,45
5,Uge 34,3,1,0,0,5,0,1,1,18,...,22,22,6,0,17,17,121,11,59,59
6,Uge 33,2,0,0,0,6,0,1,0,35,...,16,16,3,1,11,11,136,13,58,56
7,Uge 32,4,1,0,0,3,1,1,1,25,...,10,10,4,0,9,9,107,29,37,34
8,Uge 31,2,0,0,0,2,1,0,0,21,...,6,6,0,0,13,13,65,14,36,34
9,Uge 30,0,1,0,0,1,0,0,0,20,...,5,5,0,0,5,5,62,21,23,22


In [104]:
# Flip dataframe to be like df1
incidf = pd.DataFrame()
incidf_once = pd.DataFrame()
incidf_both = pd.DataFrame()
incidf_full = pd.DataFrame()
incidf_none = pd.DataFrame()

weekNames = df1.Ugenummer
incidf['Ugenummer'] = weekNames
incidf_none['Ugenummer'] = weekNames
incidf_once['Ugenummer'] = weekNames
incidf_both['Ugenummer'] = weekNames
incidf_full['Ugenummer'] = weekNames

for k in range(0,len(df3_C)):
    curRow = df3_C.loc[k]
    curAge = df3_C.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        thisNone = pd.to_numeric(thisNone.replace(',','.'))
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        thisOnce = pd.to_numeric(thisOnce.replace(',','.'))
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        thisBoth = pd.to_numeric(thisBoth.replace(',','.'))
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        thisFull = pd.to_numeric(thisFull.replace(',','.'))
        curFull.append(thisFull)
    
    # Add to dataframe
    incidf[curAge+'_Ingen vaccination'] = curNone
    incidf[curAge+'_Første vaccination'] = curOnce
    incidf[curAge+'_Anden vaccination'] = curBoth 
    incidf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    incidf_none[curAge] = curNone
    incidf_once[curAge] = curOnce
    incidf_both[curAge] = curBoth 
    incidf_full[curAge] = curFull
    

# df2_H
# curRow
incidf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 39,228.7,89.2,9.7,8.8,230.7,78.3,13.2,13.3,145.5,...,21.8,21.8,44.1,222.0,19.4,19.4,159.6,117.3,28.3,28.2
1,Uge 38,234.3,70.9,5.0,4.0,170.0,118.5,15.7,15.8,130.4,...,23.5,23.5,32.6,0.0,28.5,28.5,137.0,82.5,23.3,23.2
2,Uge 37,195.6,44.1,2.7,2.2,178.1,43.3,6.7,5.8,127.4,...,15.2,15.2,21.5,0.0,18.2,18.3,124.5,64.4,19.0,19.0
3,Uge 36,230.6,75.4,12.0,7.4,314.6,60.4,18.4,16.7,186.2,...,21.2,21.3,31.9,0.0,25.4,25.4,182.6,117.4,27.1,27.0
4,Uge 35,389.4,126.5,23.2,20.3,357.2,240.4,25.9,26.0,234.7,...,21.1,21.1,115.6,180.2,25.8,25.4,247.3,130.1,37.0,37.0
5,Uge 34,526.7,149.8,49.8,25.9,576.7,257.2,36.0,35.3,336.4,...,27.1,27.1,93.2,87.4,24.2,24.2,345.8,151.1,44.0,43.9
6,Uge 33,564.1,141.8,77.2,9.9,718.3,232.8,51.3,46.2,426.2,...,20.0,20.0,30.7,0.0,23.8,23.8,390.6,139.2,45.3,42.6
7,Uge 32,399.6,141.3,163.9,44.1,793.1,259.0,67.7,54.0,534.6,...,18.5,18.4,70.8,135.5,17.4,17.4,390.4,142.4,46.8,39.3
8,Uge 31,293.7,101.2,255.2,0.0,765.4,285.8,82.5,52.0,470.3,...,12.8,12.9,0.0,0.0,13.6,13.6,337.9,143.5,42.8,34.9
9,Uge 30,185.1,100.5,0.0,0.0,610.4,338.1,117.4,75.8,425.8,...,11.0,11.0,9.8,0.0,9.0,9.1,260.3,182.6,41.9,31.5


In [105]:
# Flip dataframe to be like df1
inciHospdf = pd.DataFrame()
inciHospdf_once = pd.DataFrame()
inciHospdf_both = pd.DataFrame()
inciHospdf_full = pd.DataFrame()
inciHospdf_none = pd.DataFrame()

weekNames = df1.Ugenummer
inciHospdf['Ugenummer'] = weekNames
inciHospdf_none['Ugenummer'] = weekNames
inciHospdf_once['Ugenummer'] = weekNames
inciHospdf_both['Ugenummer'] = weekNames
inciHospdf_full['Ugenummer'] = weekNames

for k in range(0,len(df3_H)):
    curRow = df3_H.loc[k]
    curAge = df3_H.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        thisNone = pd.to_numeric(thisNone.replace(',','.'))
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        thisOnce = pd.to_numeric(thisOnce.replace(',','.'))
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        thisBoth = pd.to_numeric(thisBoth.replace(',','.'))
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        thisFull = pd.to_numeric(thisFull.replace(',','.'))
        curFull.append(thisFull)
    
    # Add to dataframe
    inciHospdf[curAge+'_Ingen vaccination'] = curNone
    inciHospdf[curAge+'_Første vaccination'] = curOnce
    inciHospdf[curAge+'_Anden vaccination'] = curBoth 
    inciHospdf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    inciHospdf_none[curAge] = curNone
    inciHospdf_once[curAge] = curOnce
    inciHospdf_both[curAge] = curBoth 
    inciHospdf_full[curAge] = curFull
    

# df2_H
# curRow
inciHospdf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 39,0.0,0.0,0.6,0.6,0.0,19.6,0.0,0.0,5.1,...,2.8,2.8,11.0,111.0,4.6,4.6,6.0,5.2,1.0,1.0
1,Uge 38,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.9,...,3.1,3.1,21.8,0.0,9.1,9.1,5.1,2.1,1.3,1.3
2,Uge 37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4,...,2.0,2.0,0.0,0.0,6.8,6.8,5.6,4.4,1.1,1.1
3,Uge 36,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.4,...,2.6,2.6,31.9,0.0,7.6,7.6,12.8,6.1,1.5,1.5
4,Uge 35,4.7,0.0,0.0,0.0,8.2,0.0,0.0,0.0,10.5,...,2.2,2.2,94.6,180.2,6.8,6.4,18.8,6.2,1.2,1.2
5,Uge 34,3.2,1.2,0.0,0.0,13.0,0.0,0.5,0.5,10.1,...,4.0,4.0,62.1,0.0,6.4,6.4,18.1,3.5,1.5,1.7
6,Uge 33,1.9,0.0,0.0,0.0,14.9,0.0,0.5,0.0,18.7,...,2.9,2.9,30.7,78.4,4.2,4.2,19.3,2.7,1.6,1.7
7,Uge 32,3.3,0.8,0.0,0.0,7.1,3.5,0.6,0.6,12.7,...,1.8,1.8,40.5,0.0,3.4,3.4,14.3,3.8,1.1,1.1
8,Uge 31,1.3,0.0,0.0,0.0,4.5,2.1,0.0,0.0,10.0,...,1.1,1.1,0.0,0.0,4.9,4.9,7.9,1.5,1.2,1.2
9,Uge 30,0.0,1.5,0.0,0.0,2.1,0.0,0.0,0.0,8.5,...,0.9,0.9,0.0,0.0,1.9,1.9,6.7,2.0,0.8,0.9


In [108]:
allWeeks = df1['Ugenummer'].apply(lambda x: int(x[-2:]))

# Uge 36: 07-09-2021
weekOffset = allWeeks - 36
allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-07') + np.timedelta64(x*7,'D'))

# Start analysis

In [249]:
firstIndex = 5
lastIndex = 1
firstDate = allDates[firstIndex]
lastDate = allDates[lastIndex]

print(f'From {firstDate} until {lastDate}')

firstIndexDK = np.where(allDatesDK == firstDate)[0][0]
lastIndexDK  = np.where(allDatesDK ==  lastDate)[0][0]

DeathDK = allDeathDF.iloc[firstIndexDK:lastIndexDK]
print(f'Column 16 is {DeathDK.columns[16]}')
print(f'80+: {DeathDK.sum()[16:].sum()} deaths')
DeathDK.sum()


From 2021-08-24 00:00:00 until 2021-09-21 00:00:00
Column 16 is 80-84 år
80+: 2223 deaths


Age
0-4 år                17
5-9 år                 2
10-14 år               1
15-19 år               6
20-24 år               9
25-29 år               9
30-34 år               9
35-39 år              10
40-44 år              19
45-49 år              58
50-54 år              79
55-59 år             130
60-64 år             177
65-69 år             329
70-74 år             503
75-79 år             675
80-84 år             706
85-89 år             721
90-94 år             516
95-99 år             227
100 år og derover     53
dtype: int64

In [250]:
# testfilename = "C:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData\\ssi_vacc\\SSI_vacc_2021-10-05\\Vaccine_DB\\Vaccinationer_region_aldgrp_koen.csv"
# testdf = pd.read_csv(testfilename,delimiter=';',encoding='latin-1')
# numVaccOver80 = testdf[testdf.Aldersgruppe == '80-89']['Antal faerdigvacc.'].sum() + testdf[testdf.Aldersgruppe == '90+']['Antal faerdigvacc.'].sum() 
# numVaccOver80

In [251]:
vaccFilename = "C:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData\\ssi_vacc\\SSI_vacc_2021-10-05\\Vaccine_DB\\FoersteVacc_FaerdigVacc_region_aldgrp_dag.csv"
vaccdfAll = pd.read_csv(vaccFilename,delimiter=';',encoding='latin-1')
vaccdfAll.Dato = pd.to_datetime(vaccdfAll.Dato)
# numVaccOver80_2 = testdf2[testdf2.Aldersgruppe == '80-89']['Antal færdigvacc.'].sum() + testdf2[testdf2.Aldersgruppe == '90+']['Antal færdigvacc.'].sum()
# numVaccOver80_2

In [252]:
vaccdfBefore = vaccdfAll[vaccdfAll.Dato < lastDate] # Only take vaccinations until last date used
numVaccOver80Before = vaccdfBefore[vaccdfBefore.Aldersgruppe == '80-89']['Antal færdigvacc.'].sum() + vaccdfBefore[vaccdfBefore.Aldersgruppe == '90+']['Antal færdigvacc.'].sum()
numVaccOver80Before

274103

In [253]:
# PopOver80 = dkpopdf[80:].Antal.sum()
# DeathOver80 = DeathDK.sum()[16:].sum()
# PosOver80_vacc = casedf_full['80+'].iloc[lastIndex:firstIndex].sum()
# PosOver80_unvacc = casedf_none['80+'].iloc[lastIndex:firstIndex].sum()
# PosOver80_vacc
# PosOver80_unvacc

# # Calculate the number of unvaccinated people over 80
# numUnvaccOver80 = PopOver80 - numVaccOver80Before # - DeathOver80



In [254]:
# Instead, find the summary file from the specific data
curFilename = os.getcwd()+'\\ssi_vacc\\SSI_vacc_'+str(lastDate)[0:10]+'\\Vaccine_DB\\Vaccinationer_DK_aldgrp_koen.csv'
# curFilename = os.getcwd()+'\\ssi_vacc\\SSI_vacc_'+str(firstDate)[0:10]+'\\Vaccine_DB\\Vaccinationer_DK_aldgrp_koen.csv'

vaccsumdf = pd.read_csv(curFilename,delimiter=';',encoding='latin-1')
# vaccsumdf

In [255]:
# PopOver80 = dkpopdf[80:].Antal.sum()
PopOver80 = vaccsumdf[(vaccsumdf.Aldersgruppe == '80-89') | (vaccsumdf.Aldersgruppe == '90+')]['Antal borgere'].sum()
numVaccOver80 = vaccsumdf[(vaccsumdf.Aldersgruppe == '80-89') | (vaccsumdf.Aldersgruppe == '90+')]['Antal færdigvacc.'].sum()
numUnvaccOver80 = PopOver80 - numVaccOver80
DeathOver80 = DeathDK.sum()[16:].sum()
PosOver80_vacc = casedf_full['80+'].iloc[lastIndex:firstIndex].sum()
PosOver80_unvacc = casedf_none['80+'].iloc[lastIndex:firstIndex].sum()

# Estimate deaths among unvaccinated
deathRate = DeathOver80 / PopOver80
UnvaccDeaths = deathRate * numUnvaccOver80

# Number from pdf:
covPosDeathVacc = 34
covPosDeathUnvacc = 12

# Format table
print(f'--- Vaccinerede ---')
print(f'    | Alle    | Covid+ ')
print(f'------------------------')
print(f'Pers| {numVaccOver80:6.0f} | {PosOver80_vacc:6.0f} ')
print(f'------------------------')
print(f'Døde| {DeathOver80:6.0f} | ~{covPosDeathVacc:5.0f} ')
print(f'------------------------')
print(f'    | ~{100*DeathOver80/numVaccOver80:.2f}% | ~{100*covPosDeathVacc/PosOver80_vacc:.2f}%')
print(f'------------------------')
print(f'------------------------')
print(f'--- Uvaccinerede ---')
print(f'    | Alle    | Covid+ ')
print(f'------------------------')
print(f'Pers| {numUnvaccOver80:6.0f} | {PosOver80_unvacc:6.0f} ')
print(f'------------------------')
print(f'Døde| ~{UnvaccDeaths:5.0f} | ~{covPosDeathUnvacc:5.0f} ')
print(f'------------------------')
print(f'    | ~{100*UnvaccDeaths/numUnvaccOver80:.2f}% | ~{100*covPosDeathUnvacc/PosOver80_unvacc:.2f}%')


--- Vaccinerede ---
    | Alle    | Covid+ 
------------------------
Pers| 279418 |    257 
------------------------
Døde|   2223 | ~   34 
------------------------
    | ~0.80% | ~13.23%
------------------------
------------------------
--- Uvaccinerede ---
    | Alle    | Covid+ 
------------------------
Pers|  10778 |     19 
------------------------
Døde| ~   83 | ~   12 
------------------------
    | ~0.77% | ~63.16%
